In [1]:
from google.cloud import storage
import pandas as pd
import matplotlib.pyplot as plt
import json
from datetime import datetime
from urllib.parse import urlparse
import plotly.express as px
import seaborn as sns



def read_logs_from_gs_uri(gs_uri):
    """
    Read logs from a Google Cloud Storage file given a gs:// URI.
    """
    # Parse the gs:// URI
    parsed = urlparse(gs_uri)
    if parsed.scheme != "gs":
        raise ValueError(f"Invalid URI scheme: {gs_uri}. Must start with 'gs://'.")

    bucket_name = parsed.netloc
    file_path = parsed.path.lstrip('/')

    # Initialize GCS client
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(file_path)
    
    # Download the JSON content as a string
    logs_json = blob.download_as_string()

    # Parse the JSON string into a list of dictionaries
    logs = json.loads(logs_json)
    return logs

def analyze_message_flow(logs,desc = ''):
  
    # Create a DataFrame from the logs
    df = pd.DataFrame(logs)

    # Convert the 'timestamp' column to datetime type
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Sort by 'timestamp' to maintain proper order
    df = df.sort_values('timestamp')

    # Create an interactive scatter plot for the message states over time
    fig = px.line(
        df,
        x='timestamp',
        y='message_id',
        color='state',  # Color by the state to differentiate states visually
        title= desc +  "Plot for Message Transitions Over Time",
        labels={"timestamp": "Time", "message id": "Message_ID"},
        hover_data=["message_id"],
        template="plotly_dark",  # Optional: dark theme
        markers=True
    )

    # Customize layout for better readability
    fig.update_layout(
        hovermode="closest",
        xaxis=dict(showgrid=True, title="Timestamp"),
        yaxis=dict(showgrid=True, title="State"),
        margin=dict(t=40, b=40, l=40, r=40),  # Adjust margins to make space for labels
    )
    
    # Show the plot
    fig.show()




In [2]:
def calculate_latency_per_messages(logs):
  df = pd.DataFrame(logs)
  df['timestamp'] = pd.to_datetime(df['timestamp'])  # Convert to datetime
  pending_times = df[df['state'] == 'pending'].set_index('message_id')['timestamp']
  completed_times = df[df['state'] == 'completed'].set_index('message_id')['timestamp']
  latency = completed_times - pending_times
  latency_df =  latency.reset_index(name='latency')
  latency_df['latency_seconds'] = latency_df['latency'].dt.total_seconds()
  # Summary statistics
  # stats = latency_df['latency_seconds'].describe()
  # print(stats)
  return latency_df


In [3]:

def calculate_throughput(logs, time_interval='10S'):
  """
  Calculate throughput (tasks completed per time interval) from a list of logs.
  
  Args:
      logs (list): List of dictionaries containing log data. Each log should include 'timestamp' and 'state'.
      time_interval (str): Pandas offset string for time intervals (e.g., '1T' for 1 minute).
  
  Returns:
      pd.DataFrame: DataFrame with time intervals and corresponding throughput.
  """
  # Convert logs to a DataFrame
  df = pd.DataFrame(logs)
  
  # Ensure 'timestamp' is in datetime format
  df['timestamp'] = pd.to_datetime(df['timestamp'])
  
  # Filter logs for the 'completed' state
  completed_logs = df[df['state'] == 'completed']
  
  # Calculate throughput by resampling based on time intervals
  throughput = completed_logs.set_index('timestamp').resample(time_interval).size()
  
  print(f"throughput => {throughput}")
  
  # Reset index and rename the throughput column
  return throughput.reset_index(name='throughput')


In [4]:
import plotly.graph_objects as go

def plot_throughput_trends(throughput_df, desc=''):
    """
    Plots interactive throughput trends over time using Plotly.

    Args:
        throughput_df (pd.DataFrame): DataFrame with `timestamp` and `throughput` columns.
        desc (str): Description or title prefix for the plot.
    """
    fig = go.Figure()

    # Add throughput trace
    fig.add_trace(
        go.Scatter(
            x=throughput_df['timestamp'],
            y=throughput_df['throughput'],
            mode='lines+markers',
            marker=dict(color='green'),
            line=dict(color='green'),
            name='Throughput'
        )
    )

    # Update layout
    fig.update_layout(
        title=f'{desc} Plot for Throughput Trends Over Time',
        xaxis_title='Time',
        yaxis_title='Tasks Completed (Throughput)',
        template='plotly_dark',
        legend=dict(title='Legend'),
        xaxis=dict(showgrid=True),
        yaxis=dict(showgrid=True),
    )

    fig.show()

def plot_latency_trends(latency_df, desc=''):
    """
    Plots interactive latency trends over message IDs using Plotly.

    Args:
        latency_df (pd.DataFrame): DataFrame with `message_id` and `latency_seconds` columns.
        desc (str): Description or title prefix for the plot.
    """
    fig = go.Figure()

    # Add latency trace
    fig.add_trace(
        go.Scatter(
            x=latency_df['message_id'],
            y=latency_df['latency_seconds'],
            mode='lines+markers',
            marker=dict(color='blue'),
            line=dict(color='blue'),
            name='Latency'
        )
    )

    # Update layout
    fig.update_layout(
        title=f'{desc} Plot for Latency Trends Over Time',
        xaxis_title='Message ID',
        yaxis_title='Latency (seconds)',
        template='plotly_dark',
        legend=dict(title='Legend'),
        xaxis=dict(showgrid=True),
        yaxis=dict(showgrid=True),
    )

    fig.show()


In [6]:


def analyze_lentency(logs,desc):
  latency_df = calculate_latency_per_messages(logs)
  # print(latency_df)
  # plot_latency_distribution(latency_df)
  plot_latency_trends(latency_df,desc)
  

def analyze_throughtput(logs,desc):
  throughput_df = calculate_throughput(logs)
  # print(throughput_df)
  plot_throughput_trends(throughput_df,desc)
  


exps = [
  
  # ("10 tasks 2 workers batch-size => 5 : slow ","gs://cameltrain-sight/doing_mq_analysis/2024-12-20T09:00:43.541533.json"),
  # ("50 tasks 2 workers batch-size => 5 : slow ","gs://cameltrain-sight/doing_mq_analysis/2024-12-23T15:15:49.032019.json"),
  # ("100 tasks 2 workers batch-size => 5 : slow ","gs://cameltrain-sight/doing_mq_analysis/2024-12-23T19:26:50.355211.json"),
  # ("500 tasks 2 workers batch-size => 5 : slow ","gs://cameltrain-sight/doing_mq_analysis/2024-12-22T15:48:18.956719.json"),
  
  # ("50 tasks 4 workers batch-size => 5 : slow ","gs://cameltrain-sight/doing_mq_analysis/2024-12-27T09:26:30.675248.json"),
  # ("100 tasks 4 workers batch-size => 5 : slow ","gs://cameltrain-sight/doing_mq_analysis/2024-12-27T15:29:03.534160.json"),
  # ("500 tasks 4 workers batch-size => 5 : slow ","gs://cameltrain-sight/doing_mq_analysis/2024-12-22T15:48:18.956719.json"),
  
  
  # ("10 tasks 1 workers batch-size => 1 : fast ","gs://cameltrain-sight/doing_mq_analysis/2025-01-04T19:48:06.083435.json"),
  
  # ("10 tasks 2 workers batch-size => 5 : fast ","gs://cameltrain-sight/doing_mq_analysis/2024-12-31T09:40:23.962527.json"),
  # ("50 tasks 2 workers batch-size => 5 : fast ","gs://cameltrain-sight/doing_mq_analysis/2024-12-31T10:27:32.520731.json"),
  # ("100 tasks 2 workers batch-size => 5 : fast ","gs://cameltrain-sight/doing_mq_analysis/2024-12-31T10:52:02.779873.json"),
  # ("500 tasks 2 workers batch-size => 5 : fast ","gs://cameltrain-sight/doing_mq_analysis/2024-12-31T12:06:54.093683.json"), 
  
  # ("50 tasks 4 workers batch-size => 5 : fast ","gs://cameltrain-sight/doing_mq_analysis/2024-12-31T08:09:08.847832.json"), 
  # ("100 tasks 4 workers batch-size => 5 : fast ","gs://cameltrain-sight/doing_mq_analysis/2024-12-31T09:08:46.723731.json"), 
  # ("500 tasks 4 workers batch-size => 5 : fast ","gs://cameltrain-sight/doing_mq_analysis/2024-12-31T12:06:54.093683.json"),

  # # more workers

  # ("200 tasks 20 workers batch-size => 5 : fast ","gs://cameltrain-sight/doing_mq_analysis/2025-01-05T19:21:16.802607.json"),
  # ("50 tasks 25 workers batch-size => 5 : fast ","gs://cameltrain-sight/doing_mq_analysis/2025-01-05T19:34:17.502357.json"),
  
  ("1000 tasks 50 workers batch-size => 5 : fast ","gs://cameltrain-sight/doing_mq_analysis/2025-01-06T11:42:44.340375.json")
  # ('ok',"gs://cameltrain-sight/doing_mq_analysis/2025-01-08T16:14:33.056443.json")
]


for (desc,uri) in exps:
  logsx = read_logs_from_gs_uri(gs_uri = uri )
  analyze_message_flow(logsx,desc)
  analyze_lentency(logsx,desc)
  analyze_throughtput(logsx,desc)


throughput => timestamp
2025-01-06 11:38:20     2
2025-01-06 11:38:30     9
2025-01-06 11:38:40    19
2025-01-06 11:38:50    10
2025-01-06 11:39:00    19
2025-01-06 11:39:10    16
2025-01-06 11:39:20    24
2025-01-06 11:39:30    22
2025-01-06 11:39:40    22
2025-01-06 11:39:50    23
2025-01-06 11:40:00    28
2025-01-06 11:40:10    24
2025-01-06 11:40:20    52
2025-01-06 11:40:30    30
2025-01-06 11:40:40    26
2025-01-06 11:40:50    38
2025-01-06 11:41:00    45
2025-01-06 11:41:10    33
2025-01-06 11:41:20    41
2025-01-06 11:41:30    33
2025-01-06 11:41:40    37
2025-01-06 11:41:50    59
2025-01-06 11:42:00    26
2025-01-06 11:42:10    54
2025-01-06 11:42:20    43
2025-01-06 11:42:30    30
2025-01-06 11:42:40    40
Freq: 10S, dtype: int64
